In [1]:
from google.cloud import bigquery
from google.cloud import bigquery_storage
creds = "/Users/srdeo/OneDrive - Copart, Inc/secrets/cprtpr-datastewards-sp1-614d7e297848 (1).json"
client = bigquery.Client.from_service_account_json(json_credentials_path=creds)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=creds)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

In [3]:
from datetime import datetime, timedelta
import pandas as pd

# Generate list of month starts from Nov 2024 to April 2025
month_starts = pd.date_range(start='2024-11-01', end='2025-04-01', freq='MS')

query_parts = []

for start_date in month_starts:
    end_date = (start_date + pd.offsets.MonthEnd(1)).date()
    month_label = start_date.strftime('%Y-%m')
    start_str = start_date.strftime('%Y-%m-%d')
    end_str = end_date.strftime('%Y-%m-%d')

    # Combined block with CASE WHEN for churn
    query = f"""
    SELECT
        m.owner_mbr_nbr,
        m.mbr_nbr,
        m.mbr_type,
        m.mbr_mbrshp_type_cd,
        e.event_type,
        e.event_value,
        DATE_TRUNC(e.event_value, MONTH) AS event_month,
        '{month_label}' AS month_label,
        CASE
            WHEN m.mbr_type = 'LPSDPREMIER' AND e.event_type = 'MEMBER TYPE UPDATED FROM PREMIER TO GUEST' THEN 1
            WHEN m.mbr_type = 'PREMIER' AND e.event_type = 'Member Premier Renewal Paid Date' THEN 0
            ELSE NULL
        END AS churn
    FROM `cprtpr-dataplatform-sp1`.usviews.v_us_member_fact m
    JOIN `cprtpr-dataplatform-sp1`.usmart.v_us_member_events_fact e
        ON m.mbr_nbr = e.mbr_nbr
    WHERE (
        (m.mbr_type = 'PREMIER' AND e.event_type = 'Member Premier Renewal Paid Date') OR
        (m.mbr_type = 'LPSDPREMIER' AND e.event_type = 'MEMBER TYPE UPDATED FROM PREMIER TO GUEST')
    )
      AND e.event_value BETWEEN '{start_str}' AND '{end_str}'
    """

    query_parts.append(query)

# Combine all parts with UNION ALL
final_query = "\nUNION ALL\n".join(query_parts)

# Run the query
data1 = client.query(final_query).to_dataframe()

In [4]:
data1
print(data1.shape)
print(data1.isnull().sum())
print(data1['churn'].value_counts())
print(data1['month_label'].value_counts().sort_index())

,owner_mbr_nbr,mbr_nbr,mbr_type,mbr_mbrshp_type_cd,event_type,event_value,event_month,month_label,churn
0,587517,587517,PREMIER,PREMIER,Member Premier Renewal Paid Date,2025-04-20 17:23:37+00:00,2025-04-01 00:00:00+00:00,2025-04,0
1,146883,146883,PREMIER,PREMIER,Member Premier Renewal Paid Date,2025-04-29 08:04:29+00:00,2025-04-01 00:00:00+00:00,2025-04,0
2,902179,902179,PREMIER,PREMIER,Member Premier Renewal Paid Date,2025-04-28 14:06:47+00:00,2025-04-01 00:00:00+00:00,2025-04,0
3,871319,871319,PREMIER,PREMIER,Member Premier Renewal Paid Date,2025-04-09 02:47:39+00:00,2025-04-01 00:00:00+00:00,2025-04,0
4,732783,732783,PREMIER,PREMIER,Member Premier Renewal Paid Date,2025-04-15 18:09:10+00:00,2025-04-01 00:00:00+00:00,2025-04,0
...,...,...,...,...,...,...,...,...,...
42702,935114,935114,LPSDPREMIER,GUEST,MEMBER TYPE UPDATED FROM PREMIER TO GUEST,2024-11-20 18:22:09+00:00,2024-11-01 00:00:00+00:00,2024-11,1
42703,178928,178928,LPSDPREMIER,GUEST,MEMBER TYPE UPDATED FROM PREMIER TO GUEST,2024-11-23 11:00:48+00:00,2024-11-01 00:00:00+00:00,2024-11,1
42704,289664,289664,LPSDPREMIER,GUEST,MEMBER TYPE UPDATED FROM PREMIER TO GUEST,2024-11-07 11:02:14+00:00,2024-11-01 00:00:00+00:00,2024-11,1
42705,406490,406490,LPSDPREMIER,GUEST,MEMBER TYPE UPDATED FROM PREMIER TO GUEST,2024-11-20 11:00:14+00:00,2024-11-01 00:00:00+00:00,2024-11,1


(42707, 9)
owner_mbr_nbr         0
mbr_nbr               0
mbr_type              0
mbr_mbrshp_type_cd    0
event_type            0
event_value           0
event_month           0
month_label           0
churn                 0
dtype: int64
churn
0    22883
1    19824
Name: count, dtype: Int64
month_label
2024-11    5130
2024-12    8402
2025-01    8423
2025-02    6631
2025-03    7701
2025-04    6420
Name: count, dtype: int64


In [5]:
# Ensure 'month_label' is in string format like '2024-05'
unique_months = data1['month_label'].unique()

for month in unique_months:
    # Filter for that month
    df_month = data1[data1['month_label'] == month]

    # Extract lists
    churn_list = df_month[df_month['churn'] == 1]['owner_mbr_nbr'].unique().tolist()
    not_churn_list = df_month[df_month['churn'] == 0]['owner_mbr_nbr'].unique().tolist()

    # Create valid variable names: 'churn_members_may_2024', etc.
    var_suffix = pd.to_datetime(month).strftime('%B_%Y').lower()

    # Store in dynamically named lists
    globals()[f"churn_members_{var_suffix}"] = churn_list
    globals()[f"not_churn_members_{var_suffix}"] = not_churn_list


In [6]:
# Reuse the same month list from data1
unique_months = data1['month_label'].unique()

for month in unique_months:
    var_suffix = pd.to_datetime(month).strftime('%B_%Y').lower()

    # Get churn and not churn lists by name
    churn_members = globals().get(f"churn_members_{var_suffix}", [])
    not_churn_members = globals().get(f"not_churn_members_{var_suffix}", [])

    # Combine and deduplicate
    all_members = pd.unique(churn_members + not_churn_members).tolist()

    # Save as new variable
    globals()[f"all_members_{var_suffix}"] = all_members

In [7]:
# Loop through each month's member list
unique_months = data1['month_label'].unique()

for month in unique_months:
    var_suffix = pd.to_datetime(month).strftime('%B_%Y').lower()

    # Get all_members list for this month
    all_members = globals().get(f"all_members_{var_suffix}", [])

    # Convert to list of strings (no quotes)
    all_members_list = [str(x) for x in all_members]

    # Join into comma-separated string
    all_members_str = ", ".join(all_members_list)

    # Store both
    globals()[f"all_members_list_{var_suffix}"] = all_members_list
    globals()[f"all_members_str_{var_suffix}"] = all_members_str

* Below query extracts data for churned members in the last 6 months

In [54]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import numpy as np

# Full month range from August 2024 to April 2025
month_range = [
   "2025-04"
]

query_results = []

for churn_month in month_range:
    churn_dt = datetime.strptime(churn_month + "-01", "%Y-%m-%d")

    # Lookback window: 6 full calendar months before churn month
    start_dt = (churn_dt - relativedelta(months=6)).replace(day=1)
    end_dt = (churn_dt - relativedelta(days=1))  # Day before churn month

    start_str = start_dt.strftime("%Y-%m-%d")
    end_str = end_dt.strftime("%Y-%m-%d")

    month_key = churn_dt.strftime('%B').lower() + '_' + str(churn_dt.year)

    # Get churn list and all member string dynamically
    churn_members_list = globals()[f'churn_members_{month_key}']
    all_members_str = globals()[f'all_members_str_{month_key}']

    query = f"""
WITH filtered_members AS (
  SELECT
    mbr_nbr,
    owner_mbr_nbr,
    mbr_state,
    mbr_country,
    member_sgmnt_type
  FROM `cprtpr-dataplatform-sp1`.usviews.v_us_member_fact
  WHERE owner_mbr_nbr = 237640
),

bids_placed AS (
  SELECT
    fm.owner_mbr_nbr,
    fm.mbr_state,
    fm.mbr_country,
    fm.member_sgmnt_type,
    sum(
        case when l.inv_dt is not null then 1 else 0
end) AS total_bids_on_lots_sold
  FROM filtered_members fm
  LEFT JOIN `cprtpr-dataplatform-sp1`.usviews.v_us_bids_fact b
    ON fm.mbr_nbr = b.buyer_nbr
  LEFT JOIN `cprtpr-dataplatform-sp1`.usviews.v_us_lot_fact l
    ON l.lot_nbr = b.lot_nbr AND l.inv_dt BETWEEN '2024-10-01' AND '2025-03-31'
  GROUP BY fm.owner_mbr_nbr, fm.mbr_state, fm.mbr_country, fm.member_sgmnt_type
),

lots_bid_on AS (
  SELECT
    b.buyer_nbr AS owner_mbr_nbr,
    COUNT(DISTINCT b.lot_nbr) AS total_lots_placed_bids_on
  FROM `cprtpr-dataplatform-sp1`.usviews.v_us_bids_fact b
  JOIN `cprtpr-dataplatform-sp1`.usviews.v_us_lot_fact l
    ON b.lot_nbr = l.lot_nbr
  WHERE l.inv_dt BETWEEN '2024-10-01' AND '2025-03-31'
  GROUP BY b.buyer_nbr
),

lots_won AS (
  SELECT
    fm.owner_mbr_nbr,
    COUNT(DISTINCT l.lot_nbr) AS total_lots_won
  FROM filtered_members fm
  JOIN `cprtpr-dataplatform-sp1`.usviews.v_us_lot_fact l
    ON l.buyer_nbr = fm.mbr_nbr
  WHERE l.inv_dt BETWEEN '2024-10-01' AND '2025-03-31'
  GROUP BY fm.owner_mbr_nbr
),

median_sp_cte AS (
  SELECT
    fm.owner_mbr_nbr,
    APPROX_QUANTILES(l.high_bid_amt, 2)[OFFSET(1)] AS median_sp_won
  FROM filtered_members fm
  JOIN `cprtpr-dataplatform-sp1`.usviews.v_us_lot_fact l
    ON l.buyer_nbr = fm.mbr_nbr
  WHERE l.inv_dt BETWEEN '2024-10-01' AND '2025-03-31'
  GROUP BY fm.owner_mbr_nbr
),

lost_bid_stats AS (
  SELECT
    fm.owner_mbr_nbr,
    APPROX_QUANTILES(l.high_bid_amt, 2)[OFFSET(1)] AS median_sp_lost,
    MAX(b.bid_amt) AS max_bid_amt_lost
  FROM filtered_members fm
  JOIN `cprtpr-dataplatform-sp1`.usviews.v_us_bids_fact b
    ON fm.mbr_nbr = b.buyer_nbr
  JOIN `cprtpr-dataplatform-sp1`.usviews.v_us_lot_fact l
    ON b.lot_nbr = l.lot_nbr
  WHERE l.inv_dt BETWEEN '2024-10-01' AND '2025-03-31'
    AND l.buyer_nbr != fm.mbr_nbr
  GROUP BY fm.owner_mbr_nbr
)

SELECT
  b.owner_mbr_nbr,
  b.total_bids_on_lots_sold,
  lb.total_lots_placed_bids_on,
  w.total_lots_won,
  m.median_sp_won,
  l.median_sp_lost,
  l.max_bid_amt_lost,
  b.mbr_state,
  b.mbr_country,
  b.member_sgmnt_type
FROM bids_placed b
LEFT JOIN lots_bid_on lb
  ON b.owner_mbr_nbr = lb.owner_mbr_nbr
LEFT JOIN lots_won w
  ON b.owner_mbr_nbr = w.owner_mbr_nbr
LEFT JOIN median_sp_cte m
  ON b.owner_mbr_nbr = m.owner_mbr_nbr
LEFT JOIN lost_bid_stats l
  ON b.owner_mbr_nbr = l.owner_mbr_nbr
"""

    data = client.query(query).to_dataframe()

    # Post-processing
    data['churn'] = data['owner_mbr_nbr'].isin(churn_members_list).astype(int)
    #data['bid_atleast_once'] = np.where(data['total_lots_placed_bids_on'] > 0, 1, 0)
    #data['won_atleast_once'] = np.where(pd.isna(data['total_lots_won']), 0, 1)
    data.insert(0, 'month', churn_dt.strftime('%B'))

    query_results.append(data)

# Combine all results
final_df = pd.concat(query_results, ignore_index=True)

In [55]:
final_df['churn'].value_counts()

churn
1    1
Name: count, dtype: int64

In [56]:
len(all_members_april_2025)

6402

In [57]:
final_df

,month,owner_mbr_nbr,total_bids_on_lots_sold,total_lots_placed_bids_on,total_lots_won,median_sp_won,median_sp_lost,max_bid_amt_lost,mbr_state,mbr_country,member_sgmnt_type,churn
0,April,237640,0,<NA>,<NA>,NaN,NaN,NaN,OH,USA,Consumer,1


In [49]:
final_df[final_df['owner_mbr_nbr']==237640]

,month,owner_mbr_nbr,total_bids_on_lots_sold,total_lots_placed_bids_on,total_lots_won,median_sp_won,median_sp_lost,max_bid_amt_lost,mbr_state,mbr_country,member_sgmnt_type,churn
1166,April,237640,81,<NA>,<NA>,NaN,NaN,NaN,OH,USA,Consumer,1


In [50]:
final_df.isnull().sum()

month                           0
owner_mbr_nbr                   0
total_bids_on_lots_sold         0
total_lots_placed_bids_on    3067
total_lots_won               4094
median_sp_won                4094
median_sp_lost               3154
max_bid_amt_lost             3154
mbr_state                       0
mbr_country                     0
member_sgmnt_type             342
churn                           0
dtype: int64

In [38]:
final_df[final_df['median_sp_lost'].isnull()]

,month,owner_mbr_nbr,total_bids_on_lots_sold,total_lots_placed_bids_on,total_lots_won,median_sp_won,median_sp_lost,max_bid_amt_lost,mbr_state,mbr_country,member_sgmnt_type,churn
35,April,58213,19,1,1,13900.0,NaN,NaN,LV,LVA,Exporter,1
136,April,180841,23,1,1,15200.0,NaN,NaN,LA,USA,Consumer,1
160,April,791925,19,1,1,3500.0,NaN,NaN,PA,USA,Dismantler,1
195,April,254477,2,1,1,350.0,NaN,NaN,TX,USA,Consumer,0
373,April,175947,7,1,1,7900.0,NaN,NaN,CA,USA,General Business,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3210,April,629603,35,3,3,14600.0,NaN,NaN,AZ,USA,Consumer,0
3253,April,111520,3,1,1,12000.0,NaN,NaN,AB,CAN,Exporter,1
3255,April,644538,7,1,1,1300.0,NaN,NaN,MI,USA,Consumer,0
3297,April,783957,1,1,1,4100.0,NaN,NaN,FL,USA,Consumer,1
